<a href="https://colab.research.google.com/github/arthurvaz05/Python.Script/blob/master/Food_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install pulp

     |████████████████████████████████| 13.6MB 162kB/s 
  Created wheel for pulp: filename=PuLP-1.6.10-cp36-none-any.whl size=12269903 sha256=03bc8cfe0837e975b3e84509c2e444a59723649dbf4ea168b2d4215e0a5dcce2
  Stored in directory: /root/.cache/pip/wheels/5e/76/77/e28b22219e46e3b4b033f02e8b36b2770ae545bdcf60c2b224
Successfully built pulp


In [3]:
!apt-get install glpk-utils
!apt-get install coinor-cbc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 692 kB of archives.
After this operation, 1,664 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libcolamd2 amd64 1:5.1.2-2 [16.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglpk40 amd64 4.65-1 [378 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 glpk-utils amd6

In [0]:
import pandas as pd
from pulp import *
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

In [5]:
#ativando a conta
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [6]:
#verificar os arquivos disponiveis na pasta
!ls "/content/drive/My Drive/Colab Notebooks/Food_otmization"

Food_analyse.ipynb  food_base.xlsx  food_restricao.xlsx


In [0]:
df_food = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Food_otmization/food_base.xlsx')
df_food_restricao = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Food_otmization/food_restricao.xlsx')

In [8]:
df_food.head()

,Foods,Price/serving,Serving Size,Calories,Total_fat g,Carbohydrates g,Protein g,Fiber g,Cholesterol mg,Sodium mg,Calcium mg,Iron mg,Total serving g
0,Frozen Broccoli,0.48,10 OZ,73.8,0.8,13.6,8.0,8.5,0.0,68.2,159.0,2.3,31.1295
1,Frozen Corn,0.54,1/2 Cup,72.2,0.6,17.1,2.5,2.0,0.0,2.5,3.3,0.3,22.2061
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.4,0.2,0.3,0.0,1.8,3.8,0.1,0.9057
3,Baked Potatos,0.18,1/2 Cup,171.5,0.2,39.9,3.7,3.2,0.0,15.2,22.7,4.3,47.0422
4,Tofu,0.93,1/4 block,88.2,5.5,2.2,9.4,1.4,0.0,8.1,121.8,6.2,18.6361


In [9]:
df_food_restricao.head()

,Restricoes,Price/serving,Calories,Total_fat g,Carbohydrates g,Protein g,Fiber g,Cholesterol mg,Sodium mg,Calcium mg,Iron mg,Total serving g
0,Minimo,20,800,20,130,100,60,30,800,700,10,311.54
1,Maximo,35,1300,50,200,150,125,240,2000,1500,40,528.78


In [0]:
prob = LpProblem('Dieta Perfeita', LpMinimize)

In [0]:
#criar lista dos alimentos
foods_item = list(df_food['Foods'])

In [0]:
#Criar um dicionario de custo de todos os alimentos com o nome do alimento
custo = dict(zip(foods_item,df_food['Price/serving']))

In [0]:
#Criar um dicionario de calorias de todos os alimentos com o nome do alimento
calorias = dict(zip(foods_item,df_food['Calories']))

In [0]:
#Criar um dicionario de gordura de todos os alimentos com o nome do alimento
fat = dict(zip(foods_item,df_food['Total_fat g']))

In [0]:
#Criar um dicionario de carboidratos de todos os alimentos com o nome do alimento
carb = dict(zip(foods_item,df_food['Carbohydrates g']))

In [0]:
#Criar um dicionario de fibra de todos os alimentos com o nome do alimento
fiber = dict(zip(foods_item,df_food['Fiber g']))

In [0]:
#Criar um dicionario de proteina de todos os alimentos com o nome do alimento
protein = dict(zip(foods_item,df_food['Protein g']))

In [0]:
#Definindo que o modelo pode pegar qualquer valor maior que zero
food_var = LpVariable.dicts('Food', foods_item,lowBound=0,cat='Continuous')

In [0]:
#Funcao objetiva
prob += lpSum([custo[i]*food_var[i] for i in foods_item])

In [0]:
#Restricoes de calorias
prob += lpSum([calorias[f] * food_var[f] for f in foods_item]) >= 800
prob += lpSum([calorias[f] * food_var[f] for f in foods_item]) <= 1300

In [0]:
#Restricoes de Preco
prob += lpSum([custo[f] * food_var[f] for f in foods_item]) >= 20
prob += lpSum([custo[f] * food_var[f] for f in foods_item]) <= 35

In [0]:
#Restricoes de Gordura
prob += lpSum([fat[f] * food_var[f] for f in foods_item]) >= 20
prob += lpSum([fat[f] * food_var[f] for f in foods_item]) <= 50

In [0]:
#Restricoes de carboidrato
prob += lpSum([carb[f] * food_var[f] for f in foods_item]) >= 130
prob += lpSum([carb[f] * food_var[f] for f in foods_item]) <= 200

In [0]:
#Restricoes de Fibra
prob += lpSum([fiber[f] * food_var[f] for f in foods_item]) >= 60
prob += lpSum([fiber[f] * food_var[f] for f in foods_item]) <= 125

In [0]:
#Restricoes de Fibra
prob += lpSum([protein[f] * food_var[f] for f in foods_item]) >= 100
prob += lpSum([protein[f] * food_var[f] for f in foods_item]) <= 150

In [26]:
#Executar
prob.solve()

1

In [27]:
#Checar se achou uma solucao otima
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [28]:
#Verificar quanto de quais variaveis estao contidas no modelo otimo
for v in prob.variables():
  if v.varValue>0:
    print(v.name,'=',v.varValue)

Food_Frozen_Corn = 2.1741616
Food_Raw_Lettuce_Iceberg = 232.05459
Food_Roasted_Chiken = 1.9455068


In [29]:
obj = value(prob.objective)
print('Total de custo da minha dieta é: \
${}'.format(round(obj,2)))

Total de custo da minha dieta é: $20.0


In [0]:
#Problema linear inteiro
prob = LpProblem('Dieta Perfeita', LpMinimize)
food_intenger = LpVariable.dicts('Food', foods_item,0,cat = 'Integer')

In [48]:
#Funcao objetiva
prob += lpSum([custo[i]*food_intenger[i] for i in foods_item])
#Restricoes de calorias
prob += lpSum([calorias[f] * food_intenger[f] for f in foods_item]) >= 800
prob += lpSum([calorias[f] * food_intenger[f] for f in foods_item]) <= 1300
#Restricoes de Preco
prob += lpSum([custo[f] * food_intenger[f] for f in foods_item]) >= 20
prob += lpSum([custo[f] * food_intenger[f] for f in foods_item]) <= 35
#Restricoes de Gordura
prob += lpSum([fat[f] * food_intenger[f] for f in foods_item]) >= 20
prob += lpSum([fat[f] * food_intenger[f] for f in foods_item]) <= 50
#Restricoes de carboidrato
prob += lpSum([carb[f] * food_intenger[f] for f in foods_item]) >= 130
prob += lpSum([carb[f] * food_intenger[f] for f in foods_item]) <= 200
#Restricoes de Fibra
prob += lpSum([fiber[f] * food_intenger[f] for f in foods_item]) >= 60
prob += lpSum([fiber[f] * food_intenger[f] for f in foods_item]) <= 125
#Restricoes de Fibra
prob += lpSum([protein[f] * food_intenger[f] for f in foods_item]) >= 100
prob += lpSum([protein[f] * food_intenger[f] for f in foods_item]) <= 150
#Executar
prob.solve()
#Checar se achou uma solucao otima
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [49]:
#Verificar quanto de quais variaveis estao contidas no modelo otimo
for v in prob.variables():
  if v.varValue>0:
    print(v.name,'=',v.varValue)

Food_Frozen_Broccoli = 3.0
Food_Raw_Lettuce_Iceberg = 232.0
Food_Tofu = 5.0


In [50]:
obj = value(prob.objective)
print('Total de custo da minha dieta é: \
${}'.format(round(obj,1)))

Total de custo da minha dieta é: $20.0


In [0]:
#Aumentando a complexidade com restricao inteira
prob = LpProblem('Dieta Perfeita', LpMinimize)
food_intenger = LpVariable.dicts('Porção', foods_item,0,cat = 'Continuous')

In [0]:
#Restricao inteira
food_chosen = LpVariable.dicts('Chosen', foods_item,0,1,cat='Integer')

In [0]:
#Funcao objetiva
prob += lpSum([custo[i]*food_intenger[i] for i in foods_item])
#Restricoes de calorias
prob += lpSum([calorias[f] * food_intenger[f] for f in foods_item]) >= 800
prob += lpSum([calorias[f] * food_intenger[f] for f in foods_item]) <= 1300
#Restricoes de Gordura
prob += lpSum([fat[f] * food_intenger[f] for f in foods_item]) >= 20
prob += lpSum([fat[f] * food_intenger[f] for f in foods_item]) <= 50
#Restricoes de carboidrato
prob += lpSum([carb[f] * food_intenger[f] for f in foods_item]) >= 130
prob += lpSum([carb[f] * food_intenger[f] for f in foods_item]) <= 200
#Restricoes de Fibra
prob += lpSum([fiber[f] * food_intenger[f] for f in foods_item]) >= 60
prob += lpSum([fiber[f] * food_intenger[f] for f in foods_item]) <= 125
#Restricoes de Fibra
prob += lpSum([protein[f] * food_intenger[f] for f in foods_item]) >= 100
prob += lpSum([protein[f] * food_intenger[f] for f in foods_item]) <= 150

In [0]:
#adicionando as resticoes inteiras
for f in foods_item:
  prob += food_intenger[f]>= food_chosen[f]*0.1
  prob += food_intenger[f]<= food_chosen[f]*1e5

In [0]:
prob += food_chosen['Frozen Broccoli'] + food_chosen['Tofu'] <= 1

In [0]:
protein_chosen = ['Roasted Chiken','Beef Frankfurter','Turkey Bologna']

In [0]:
prob += lpSum([food_chosen[p] for p in protein_chosen]) >= 1

In [0]:
prob.writeLP('Dieta_restrita.txt')

In [42]:
prob.solve()

1

In [43]:
#Checar se achou uma solucao otima
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [44]:
#Verificar quanto de quais variaveis estao contidas no modelo otimo
for v in prob.variables():
  if v.varValue>0 and v.name[0]=='P':
    print(v.name,'=',v.varValue)

Porção_Baked_Potatos = 1.2507587
Porção_Frozen_Broccoli = 6.5879497
Porção_Roasted_Chiken = 0.1
Porção_Scrambled_Eggs = 5.7385963


In [45]:
obj = value(prob.objective)
print('Total de custo da minha dieta é: \
${}'.format(round(obj,2)))

Total de custo da minha dieta é: $5.53
